<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#버전-2까지-수행하고-난-후의-개선사항" data-toc-modified-id="버전-2까지-수행하고-난-후의-개선사항-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>버전 2까지 수행하고 난 후의 개선사항</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#데이터-로드-및-FE-함수" data-toc-modified-id="데이터-로드-및-FE-함수-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>데이터 로드 및 FE 함수</a></span></li><li><span><a href="#구분끼리-유형-나누어서-예측해보기" data-toc-modified-id="구분끼리-유형-나누어서-예측해보기-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>구분끼리 유형 나누어서 예측해보기</a></span></li><li><span><a href="#다음에-할-일" data-toc-modified-id="다음에-할-일-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>다음에 할 일</a></span></li></ul></li></ul></li></ul></div>

# 버전 2까지 수행하고 난 후의 개선사항
- 날씨 데이터 사용의 효과가 그닥.. -> 날씨 데이터 그만파자.. 오히려 빼는게 좋을 수도.. -> 자기상관변수 증감률 데이터 FE하기!
- 모든 데이터를 다 사용했을 때가 Test 데이터에 대해 리더보드 결과는 더 좋음..? 두 가지 경우 모두 테스트 해야 할듯..
- 다른 모델들 사용 -> 여러가지 모델...뭘 사용할까!?
- 함수화 할것
    - 데이터 로드 및 FE 함수
    - 모델링 후 학습, 검증 결과 출력하는 함수
    - 모델링 학습 후 최종 Test 출력하는 함수
    - 제출하는 csv 파일 만드는 함수

### 데이터 로드 및 FE 함수
- 날씨 데이터는 빼고 시도..
- 증감률 추가해야 함

In [62]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pymysql
import warnings
warnings.filterwarnings(action='ignore')


def load_datasets(path) -> pd.DataFrame:
    train = pd.read_csv(os.path.join(path, '한국가스공사_시간별 공급량_20181231.csv'),
                        encoding='cp949')
    test = pd.read_csv(os.path.join(path, 'test.csv'))
    # test 데이터 전처리
    test['연월일'] = test['일자|시간|구분'].str.split(' ', expand=True)[0]
    test['시간'] = test['일자|시간|구분'].str.split(' ', expand=True)[1].astype(int)
    test['구분'] = test['일자|시간|구분'].str.split(' ', expand=True)[2]
    del test['일자|시간|구분']

    data = pd.concat([train, test], axis=0)
    data['연월일'] = pd.to_datetime(data['연월일'])
    return data

def load_weather() -> pd.DataFrame:
    db = pymysql.connect(host='localhost', port=3306, user='younghun', password='watson1259',
                        db='dacon_gas_weather_db', charset='utf8')
    cursor = db.cursor()
    sql = "SELECT datetime, avg_temp, min_temp, max_temp,\
              NULLIF(sum_rain, '') as sum_rain, avg_wind, avg_humid,\
              sum_gsr, NULLIF(ddmefs, '') as ddmefs, avg_ts \
              FROM weather ORDER BY datetime"
    
    weather = pd.read_sql(sql, db)
    weather = weather.fillna(0.)
    
    return weather
    
def merge_gas_weather(path):
    gas = load_datasets(path)
    weather = load_weather()
    gas_weather = gas.merge(weather, how='left', left_on='연월일', right_on='datetime')
    del gas_weather['datetime']
    
    return gas_weather

def make_datetime_vars(data) -> pd.DataFrame:
    data['year'] = data['연월일'].dt.year
    data['month'] = data['연월일'].dt.month
    data['day'] = data['연월일'].dt.day
    data['week_no'] = data['연월일'].dt.strftime("%V").astype(int)
    data['dayofweek'] = data['연월일'].dt.day_name()
    data['weekend_yn'] = np.where(data['dayofweek'].isin(['Saturday', 'Sunday']), 1, 0)
    data['dayofyear'] = data['연월일'].dt.dayofyear

    return data


def change_dates(row):
    if row['시간'] == 24:
        row['연월일'] += pd.DateOffset(days=1)
        row['시간'] = 0
    return row


def extract_need_data(df):
    """ 연월일 날짜 포맷 바꾸고 매년 01-01~03-31 기간 데이터만 추출"""
    df = df.apply(change_dates, axis=1)
    df['시간'] = df['시간'].apply(lambda x: str(x) if x >= 10 else '0' + str(x))
    df['연월일'] = df['연월일'].astype(str)
    df['datetime'] = df['연월일'] + ' ' + df['시간']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

    df['prev_datetime'] = df['datetime'] - pd.DateOffset(years=1)

    return df


def make_autocorr_vars(df, gubun='A'):
    sub_df = df[df['구분'] == gubun]
    # 일년 전 그 시간대 공급량
    sub_df['prev_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                          sub_df['datetime'].dt.hour])['공급량'].shift()
#     # 이년 전 공급량 파생변수 생성
#     sub_df['prev_two_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
#                                               sub_df['datetime'].dt.hour])['공급량'].shift(2)
    # 일년 전 평균 공급량
    sub_df['prev_year'] = sub_df['prev_datetime'].dt.year
    prev_year_mean_df = sub_df.groupby([sub_df['datetime'].dt.year])['공급량'].mean()
    prev_year_mean_dict = dict(zip(prev_year_mean_df.index, prev_year_mean_df.values.reshape(-1, )))
    sub_df['prev_year_avg_공급량'] = sub_df['prev_year'].map(prev_year_mean_dict)
    # 일년 전 월별 평균 공급량
    prev_month_mean_df = sub_df.groupby([sub_df['datetime'].dt.year, sub_df['datetime'].dt.month])['공급량'].mean()
    prev_month_mean_dict = dict(zip(prev_month_mean_df.index, prev_month_mean_df.values.reshape(-1, )))
    sub_df['prev_year_month'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                        sub_df['prev_datetime'].dt.month.values.reshape(-1, )))
    sub_df['prev_month_avg_공급량'] = sub_df['prev_year_month'].map(prev_month_mean_dict)
    # 일년 전 월-일자별 평균 공급량
    prev_month_day_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                           sub_df['datetime'].dt.month,
                                           sub_df['datetime'].dt.day])['공급량'].mean()
    prev_month_day_mean_dict = dict(zip(prev_month_day_mean_df.index, prev_month_day_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_day'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.day.values.reshape(-1, )))
    sub_df['prev_month_day_avg_공급량'] = sub_df['prev_year_month_day'].map(prev_month_day_mean_dict)
    # 일년 전 월-시간 별 평균 공급량
    prev_month_hour_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                            sub_df['datetime'].dt.month,
                                            sub_df['datetime'].dt.hour])['공급량'].mean()
    prev_month_hour_mean_dict = dict(zip(prev_month_hour_mean_df.index, prev_month_hour_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_hour'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.hour.values.reshape(-1, )))
    sub_df['prev_month_hour_avg_공급량'] = sub_df['prev_year_month_hour'].map(prev_month_hour_mean_dict)
    # 일년 전 그 날의 기상 변수 FE
    weather_cols = ['avg_temp','min_temp','max_temp','sum_rain','avg_wind','avg_humid','sum_gsr','ddmefs','avg_ts']
    for col in weather_cols:
        sub_df[f'prev_year_{col}'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                                  sub_df['datetime'].dt.hour])[col].shift()
        sub_df[f'prev_year_{col}'] = pd.to_numeric(sub_df[f'prev_year_{col}'], downcast="float")

    # 필요한 칼럼들만 추출
#     used_cols = ['연월일', '시간', 'datetime', '구분', 'prev_year_공급량',
#                  'prev_two_year_공급량', 'prev_year_avg_공급량', 'prev_month_avg_공급량', 'prev_month_day_avg_공급량',
#                  'prev_month_hour_avg_공급량', '공급량']
    used_cols = ['연월일', '시간', 'datetime', '구분', 'prev_year_공급량', 'prev_year_avg_공급량',
                 'prev_month_avg_공급량', 'prev_month_day_avg_공급량', 'prev_month_hour_avg_공급량']
    weather_cols = [f'prev_year_{col}' for col in weather_cols]
    used_cols += weather_cols + ['공급량']
    sub_df = sub_df[used_cols]
    return sub_df


def fe_autocorr_vars(df):
    final_df = pd.DataFrame()
    gubun_cols = ['A','B','C','D','E','G','H']
    for gubun in tqdm(gubun_cols):
        sub_df = make_autocorr_vars(df, gubun=gubun)
        final_df = pd.concat([final_df, sub_df], axis=0)
    return final_df

def change_dates_adversely(row):
    if row['datetime'].hour == 0:
        row['연월일'] -= pd.DateOffset(days=1)
        row['시간'] = 24
    return row

def change_date_format(df):
    df['연월일'] = pd.to_datetime(df['연월일'])
    df['시간'] = df['시간'].astype(int)
    df = df.apply(change_dates_adversely, axis=1)
    
    del df['datetime']
    
    return df


path = '/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas'
gas_weather = merge_gas_weather(path)
dataset = make_datetime_vars(gas_weather)
dataset = extract_need_data(dataset)
final_df = fe_autocorr_vars(dataset)
final_df = change_date_format(final_df)
final_df = final_df.reset_index(drop=True)

print(final_df.shape)




100%|██████████| 7/7 [00:07<00:00,  1.03s/it]


(383208, 18)


In [110]:
# 2014년부터 데이터 추출하는데, 매변 01-03월만 사용할지, 전체 사용할지 함수
def extract_dataset(final_df, split=False):
    # 2014-01-01부터 사용
    cond = final_df['연월일'].dt.year >= 2014
    final_df = final_df[cond]
    if split:
        cond0103 = (final_df['연월일'].dt.month >= 1) & (final_df['연월일'].dt.month <= 3)
        final_df = final_df[cond0103]
        return final_df
    else:
        return final_df
    
# 구분 없이 통합 예측 or 구분 유형대로 예측하기 위한 데이터 포맷 형태
def predict_simultaneously(final_df, simultaneously=False):
    if simultaneously:
        # 구분 원-핫 인코딩
        final_df[['B','C','D','E','G','H']] = pd.get_dummies(final_df['구분'], drop_first=True).values
        cols_order = final_df.columns.tolist()
        cols_order.remove('공급량')
        cols_order.insert(len(cols_order), '공급량')
        final_df = final_df[cols_order]
        return final_df
    else:
        return final_df

### 구분끼리 유형 나누어서 예측해보기

In [132]:
# 구분 유형별로 predict
def get_NMAE(true, pred):
    mae = np.abs(true-pred)/true
    nmae = np.mean(mae)
    return nmae

def predict_each_gubun(final_df, model, submission_csv, simultaneously=True):
    # 통합해서 예측
    if simultaneously:
        final_df = final_df.set_index(['연월일', '시간', '구분'])
        idx_level = final_df.index.get_level_values
        #======================
        # Train, Valid로 성능 검증
        #======================
        train = final_df[(idx_level(0) < '2018-01-01')]
        train = train.fillna(method='ffill')
        valid = final_df[(idx_level(0) >=' 2018-01-01') & (idx_level(0) <= '2018-03-31')]
        # X, y 분할
        X_train, y_train = train.iloc[:, :-1].values, train['공급량']
        X_valid, y_valid = valid.iloc[:, :-1].values, valid['공급량']
        # fit
        model.fit(X_train, y_train)
        # predict
        train_pred = model.predict(X_train)
        valid_pred = model.predict(X_valid)
        # evaluate
        train_nmae = get_NMAE(y_train, train_pred)
        valid_nmae = get_NMAE(y_valid, valid_pred)
        print(f'# 통합 예측 - Train NAME:', round(train_nmae, 3))
        print(f'# 통합 예측 - Valid NMAE:', round(valid_nmae, 3))
        #======================
        # 모두 학습시키고 Test로 예측
        #======================
        train = final_df[(idx_level(0) < '2019-01-01')]
        train = train.fillna(method='ffill')
        test = final_df[(idx_level(0) >= '2019-01-01')]
        # X,y 분할
        X_train, y_train = train.iloc[:, :-1].values, train['공급량']
        X_test = test.iloc[:, :-1].fillna(method='ffill').values
        # fit
        model.fit(X_train, y_train)
        # predict
        test_pred = model.predict(X_test)
        submission_csv['공급량'] = test_pred
        return submission_csv
        
    else:
        cols = ['A', 'B', 'C', 'D', 'E', 'G', 'H']
        total_test_pred = np.array([])
        for col in cols:
            gubun_df = final_df[final_df['구분'] == col]
            gubun_df = gubun_df.set_index(['연월일','시간','구분'])

            idx_level = gubun_df.index.get_level_values
            #======================
            # Train, Valid로 성능 검증
            #======================
            train = gubun_df[(idx_level(0) < '2018-01-01')]
            train = train.fillna(method='ffill')  # 결측치 있으면 안도는 모델용
            valid = gubun_df[(idx_level(0) >= '2018-01-01') & (idx_level(0) <= '2018-03-31')]
            # X, y 분할
            X_train, y_train = train.iloc[:, :-1].values, train['공급량']
            X_valid, y_valid = valid.iloc[:, :-1].values, valid['공급량']
            # fit
            model.fit(X_train, y_train)
            # predict
            train_pred = model.predict(X_train)
            valid_pred = model.predict(X_valid)
            # evaluate
            train_nmae = get_NMAE(y_train, train_pred)
            valid_nmae = get_NMAE(y_valid, valid_pred)
            print(f'# {col} - Train NAME:', round(train_nmae, 3))
            print(f'# {col} - Valid NMAE:', round(valid_nmae, 3))
            print()
            #======================
            # 모두 학습시키고 Test로 예측
            #======================
            train = gubun_df[(idx_level(0) < '2019-01-01')]
            train = train.fillna(method='ffill')
            test = gubun_df[(idx_level(0) >= '2019-01-01')]
            # X,y 분할
            X_train, y_train = train.iloc[:, :-1].values, train['공급량']
            X_test = test.iloc[:, :-1]
            X_test = X_test.fillna(method='ffill')  # 결측치 있으면 안도는 모델용
            X_test = X_test.values
            # fit
            model.fit(X_train, y_train)
            # predict
            test_pred = model.predict(X_test)
            total_test_pred = np.append(total_test_pred, test_pred)

        submission_csv['공급량'] = total_test_pred
        return submission_csv, test

In [133]:
# model
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor


# 01-03월만 추출
df = extract_dataset(final_df, split=False)
# 구분 분리안하고 동시에 예측
df = predict_simultaneously(df, simultaneously=False)

# submit
submission_csv = pd.read_csv('/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas/sample_submission.csv')
# model
lgbm = LGBMRegressor(n_estimators=100)
xgb = XGBRegressor(n_estimators=100)
voting_reg = VotingRegressor(estimators=[('lgbm', lgbm),
                                        ('xgb', xgb)])

submission_csv, test = predict_each_gubun(final_df=df, model=voting_reg, submission_csv=submission_csv,
                                          simultaneously=False)

[15:49:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# A - Train NAME: 0.107
# A - Valid NMAE: 0.16

[15:49:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:49:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# B - Train NAME: 0.116
# B - Valid NMAE: 0.187

[15:49:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:49:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# C - Train NAME: 1.104
# C - Valid NMAE: 0.141

[15:49:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:49:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# D - Train NAME: 0.109
# D - Valid NMAE: 

In [131]:
sub_path = '/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas/submission/'

# 파일명 -> 01-03월 분리여부_구분유형분리여부_모델종류_trainNMAE_validNMAE.csv
submission_csv.to_csv(sub_path+'0103True_SimulFalse_xgb+lgbm.csv', index=False)

- 01-03월: False, 통합예측 : False, XGB+LGBM => 1.5->1.40로 성능 확 개선!
- 01-03월: True, 통합예측 : False, XGB+LGBM => 1.5로 다시 늘어남 => 따라서 예측은 통합예측으로 수행하자!

### 다음에 할 일
- **시계열 데이터는 모두 사용하기(split=False)** (01-03월만 추출 X)
- **구분유형 분리해서 개별로 예측(simultaneously=False)**
- 현재 사용한 변수:
    - 자기상관 변수
        - 1년 전 그시간 공급량
        - 1년 전 연도별 평균 공급량
        - 1년 전 연-월별 평균 공급량
        - 1년 전 연-월-일자별 평균 공급량
        - 1년 전 연-월-시간대별 평균 공급량
    - 날씨 변수
        - 1년 전 평균 기온
        - 1년 전 최저 기온
        - 1년 전 최고 기온
        - 1년 전 강수합계량
        - 1년 전 평균풍속
        - 1년 전 평균상대습도
        - 1년 전 평균합계일사량
        - 1년 전 평균신적설
        - 1년 전 평균지면온도

In [137]:
# 자기상관 변수 더 만들자..! => 변화율(ratio)
# 1. 일년전 그 년도의 평균 공급량 대비 일년 전 공급량이 얼마나 증가했는지?
# 2. 일년전 그 년도의 월별 평균 공급량 대비 일년 전 공급량이 얼마나 증가했는지?
# 3. 일년전 그 년도의 월-일자별 평균 공급량 대비 일년 전 공급량이 얼마나 증가했는지?
# 4. 일년전 그 년도의 월-시간별 평균 공급량 대비 일년 전 공급량이 얼마나 증가했는지?
### 우선 위 변수들 FE하고 성능 테스트하기 -> 만약 성능 떨어진다면 LGBM으로 한번 모델링하고 feature Importance보고 날씨 변수 drop여부 결정 
### -> drop하게 된다면 drop하고 재 모델링